In [1]:
import sys, os, pickle
import torch
sys.path.append('/home/om2382/mft-theory/')
from cluster import *
from core import *
from empirics import *
from functions import *
from ode_methods import *
from plotting import *
from theory import *
from utils import *
import matplotlib.pyplot as plt

In [2]:
### --- SET UP ALL CONFIGS --- ###
from itertools import product
n_seeds = 2
macro_configs = config_generator(g=[1.3, 1.6, 1.7])

micro_configs = tuple(product(macro_configs, list(range(n_seeds))))
prototype = False

### --- SELECT PARTICULAR CONFIG --- ###
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
    prototype = True
params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds

new_random_seed_per_condition = True
if new_random_seed_per_condition:
    np.random.seed(i_job)
else: #Match random seeds across conditions
    np.random.seed(i_seed)

In [3]:
### --- Generate J matrix --- ###

N = 4000
g = params['g']
W = g*np.random.normal(0, 1/np.sqrt(N), (N, N))

In [4]:
### --- Measure covariance spectra --- ###

W_ = torch.from_numpy(W).type(torch.FloatTensor).to(0)
dt = 0.05
dt_save = 1
phi_torch = lambda x: torch.erf((np.sqrt(np.pi)/2)*x)
eigs_x, eigs_r = estimate_cov_eigs(3000, dt_save, dt, W_, phi_torch=phi_torch,
                                   T_save_delay=100, N_batch=1, N_loops=5, return_vecs=True)

KeyboardInterrupt: 

In [ ]:
### --- Observe activity --- ###
if True:
    W_ = torch.from_numpy(W).type(torch.FloatTensor).to(0)
    phi_torch = lambda x: torch.erf((np.sqrt(np.pi)/2)*x)
    x, r = sample_activity(5000, 0.1, 0.05, W_, phi_torch=phi_torch,
                           N_batch=1, T_save_delay=100)
    

In [ ]:
plt.plot(np.arange(0, 4900, 0.1), x[:,0])
#plt.xlabel('time constants')
#plt.ylim([-1, 1])

In [ ]:
#Delta_T_emp = np.fft.irfft(np.square(np.abs(np.fft.rfft(x[10000:,:1000], axis=0, norm='ortho'))).mean(1))

In [ ]:
#time_emp = np.arange(0, 100, 0.1)
#plt.plot(time_emp, Delta_T_emp[:len(time_emp)])

In [ ]:
#d = compute_Delta_0(g=g)
#time, Delta_T = integrate_potential(d, g=g, tau_max=100, N_tau=10000)
#Delta_T = fix(Delta_T)
#C_Phi_half = compute_C_simple(d, Delta_T)
#alpha = compute_phi_prime_avg(d)

In [ ]:
#time_emp = np.arange(0, 100, 0.1)
#plt.plot(time_emp, Delta_T_emp[:len(time_emp)])
#time_theory = np.arange(0, 100, 0.01)
#plt.plot(time_theory, Delta_T)

In [ ]:
#%matplotlib notebook
#plt.plot(sorted(eigs_x)[::-1], '.')
#plt.xscale('log')
#plt.yscale('log')

In [ ]:
#plt.plot(sorted(eigs_r)[::-1])
#plt.xscale('log')
#plt.yscale('log')

In [ ]:
eigs_r_ = np.zeros(5000)
eigs_r_[:N] = np.array(sorted(np.abs(eigs_r))[::-1])
eigs_x_ = np.zeros(5000)
eigs_x_[:N] = np.array(sorted(np.abs(eigs_x))[::-1])
processed_data = np.array([eigs_x_, eigs_r_])

In [ ]:
### --- SAVE RESULTS -- ###
result = {'sim': None, 'i_seed': i_seed, 'config': params,
          'i_config': i_config, 'i_job': i_job}
try:
    result['processed_data'] = processed_data
except NameError:
    pass
    
try:
    save_dir = os.environ['SAVEDIR']
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, 'result_{}'.format(i_job))

    with open(save_path, 'wb') as f:
        pickle.dump(result, f)
except KeyError:
    pass

In [ ]:
###Truncate file above
file_name = 'J_power_spectra'
job_name = 'phase_transition_power_3'
project_dir = '/home/om2382/low-rank-dims/'
main_script_path = os.path.join(project_dir, 'cluster_main_scripts', job_name + '.py')
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py {}'.format(file_name, main_script_path))

In [ ]:
###Submit job to cluster
n_jobs = len(micro_configs)
write_job_file(job_name, py_file_name='{}.py'.format(job_name), mem=32, n_hours=1, n_gpus=1)
job_script_path = os.path.join(project_dir, 'job_scripts', job_name + '.s')
submit_job(job_script_path, n_jobs, execute=False)

In [ ]:
###Get job status
get_ipython().system('squeue -u om2382')

In [ ]:
project_dir = '/home/om2382/low-rank-dims/'
job_name = 'phase_transition_power_3'
job_script_path = os.path.join(project_dir, 'job_scripts', job_name + '.s')
configs_array, results_array, key_order, sim_dict = unpack_processed_data(job_script_path)

In [ ]:
!ls -t ../job_scripts/

In [ ]:
from scipy.stats import linregress
fig, ax = plt.subplots(3, 2, figsize=(7, 7))
i_seed = 0
i_xr = 0
r_shift = 10
N = 5000
colors = [('0.5'), 'C3']
for i in range(3):
    for j in range(1):
        ax[i,0].set_ylabel('g = {}'.format(configs_array['g'][i]))
        #ax[0,j].set_title('N = {}'.format(configs_array['N'][j]))
        for k in range(i_seed+1):
            #Plot spectrum and best fit up to cutoff
            s = results_array[i,k,i_xr,:]
            N_cutoff = int(0.15*N)
            #r_shift = 10
            logs = np.log10(s)[r_shift:N_cutoff]
            x = np.arange(r_shift, len(logs) + r_shift)
            logx = np.log10(x)
            res = linregress(logx, logs)

            ax[i,j].plot(s, '.', color=colors[i_xr])
            ax[i,j].plot(x, 10**(res.intercept) * x**(res.slope), color='k')
            #ax[i,j].set_ylim([0.01, 1000])
            if k==0:
                ax[i,j].legend(['_', r'$\beta = {}$'.format(np.round(res.slope, 2))])
            ax[i,j].set_xscale('log')
            ax[i,j].set_yscale('log')
            ax[i,j].set_xticks([])
            ax[i,j].axvline(x=N_cutoff, color='k', linestyle='--')
            ax[i,j].axvline(x=r_shift, color='k', linestyle='--')
#fig.savefig('figs/x_spectra_transition.pdf')

In [ ]:
from scipy.stats import linregress
fig, ax = plt.subplots(3, 3, figsize=(9, 9))
i_seed = 0
i_xr = 0
r_shift = 10
colors = [('0.5'), 'C3']
for i in range(3):
    for j in range(3):
        ax[i,0].set_ylabel('g = {}'.format(configs_array['g'][i]))
        ax[0,j].set_title('N = {}'.format(configs_array['N'][j]))
        for k in range(i_seed+1):
            #Plot spectrum and best fit up to cutoff
            s = results_array[i,j,k,i_xr,:]
            N_cutoff = int(0.1*configs_array['N'][j])
            r_shift = 10
            logs = np.log10(s)[r_shift:N_cutoff]
            x = np.arange(r_shift, len(logs) + r_shift)
            logx = np.log10(x)
            res = linregress(logx, logs)

            ax[i,j].plot(s, '.', color=colors[i_xr])
            ax[i,j].plot(x, 10**(res.intercept) * x**(res.slope), color='k')
            #ax[i,j].set_ylim([0.01, 1000])
            if k==0:
                ax[i,j].legend(['_', r'$\beta = {}$'.format(np.round(res.slope, 2))])
            ax[i,j].set_xscale('log')
            ax[i,j].set_yscale('log')
            ax[i,j].set_xticks([])
            ax[i,j].axvline(x=N_cutoff, color='k', linestyle='--')
fig.savefig('figs/x_spectra_transition.pdf')

In [ ]:
n_x, n_y, n_seed, _, N = results_array.shape
fig, ax = plt.subplots(n_x, n_y, figsize=(15, 15))
l = 0
idx = np.logspace(0, np.log10(N), 300).astype(int)
idx = np.array(sorted(list(set(idx)))) - 1
for i in range(n_x):
    for j in range(n_y):
        for k in range(n_seed):
            if i == 0:
                ax[i,j].set_title('rho = {}'.format(np.round(configs_array[key_order[1]][j], 2)))
            if j == 0:
                ax[i,j].set_ylabel('gamma = {}'.format(np.round(configs_array[key_order[0]][i],2)))
            #Plot spectrum and best fit up to cutoff
            s = results_array[i,j,k,l,:]
            N_cutoff = int(0.1*N)
            ax[i,j].plot(idx, s[idx], '.', color='C{}'.format(k), alpha=0.8, markersize=0.5)
            ax[i,j].set_ylim([0.00001, 1000])
            ax[i,j].set_xlim([1, N])
            ax[i,j].set_xscale('log')
            ax[i,j].set_yscale('log')
            if i !=0 or j != 0:
                ax[i,j].set_xticks([])
            ax[i,j].axvline(x=N_cutoff, color='k', linestyle='--')
    #ax[0,0].legend(['Cov spec', '_fits', 'PRV'])
#fig.savefig('figs/LDRG_zscored_spec.pdf')

In [ ]:
fig, ax = plt.subplots(6, 7, figsize=(17, 17))
N = 4000
l = 0
idx = np.logspace(0, np.log10(N), 300).astype(int)
idx = np.array(sorted(list(set(idx)))) - 1
for i in range(6):
    for j in range(7):
        for k in range(5):
            if i == 0:
                G = np.exp(-configs_array['beta'][j]*np.arange(N)/N)
                PR_G = np.sum(G)**2 / np.sum(G**2) / N
                ax[i,j].set_title('$PR_G = {}$'.format(np.round(PR_G, 2)))
            if j == 0:
                ax[i,j].set_ylabel('alpha = {}'.format(np.round(configs_array['alpha'][i],2)))
            #Plot spectrum and best fit up to cutoff
            s = results_array[i,j,k,l,:]
            N_cutoff = int(configs_array['alpha'][i]*N)
            ax[i,j].plot(idx, s[idx], '.', color='C{}'.format(k), alpha=0.8, markersize=0.5)
            ax[i,j].set_ylim([0.00001, 1000])
            ax[i,j].set_xlim([1, N*5])
            ax[i,j].set_xscale('log')
            ax[i,j].set_yscale('log')
            if i !=0 or j != 0:
                ax[i,j].set_xticks([])
            ax[i,j].axvline(x=N_cutoff, color='k', linestyle='--')
    #ax[0,0].legend(['Cov spec', '_fits', 'PRV'])
fig.savefig('figs/LDRG_zscored_spec.pdf')

In [ ]:
### --- NUMERICAL FIT --- ###

